<div style="text-align: right">Paul Novaes<br>July 2018</div> 

# PRIMES in NP

This notebook shows that primes have efficient short proofs of primality.

More precisely, for any prime $p$, there is a certificate of primality whose length is polynomial in the number of digits of $p$, and that can be verified in polynomial time.

Note that we are not saying that it is easy to find such certificates. We are just saying that they exist.

## PRIMES is in coNP

A related problem is to show that a number is __not__ prime. This is a much easier problem: to show that $n$ is composite it is enough to give 2 integers $a$ and $b$, greater than 1, such that 

$$n = ab$$

Here the certificate of compositeness is the pair $(a, b)$ and it can be verified by computing $ab$ and checking that it gives $n$, indeed. For example:

In [1]:
# These Mersenne numbers are known to be prime.
mersenne_521 = 2**521 - 1
mersenne_607 = 2**607 - 1

n = mersenne_521 * mersenne_607
a = mersenne_521
b = mersenne_607

print('(', a, ',', b, ')')
print('\nis a certificate of compositeness for\n')
print(n)

( 6864797660130609714981900799081393217269435300143305409394463459185543183397656052122559640661454554977296311391480858037121987999716643812574028291115057151 , 531137992816767098689588206552468627329593117727031923199444138200403559860852242739162502265229285668889329486246501015346579337652707239409519978766587351943831270835393219031728127 )

is a certificate of compositeness for

3646154850295011369707131011438711095400799139943170490872585628683549034362552065955809589514611470241298944167703929337528884908857116141935206466329731087514964112054543019336536216107629523597606330154669196064144182472739556974502462402438903115845725630946428943768540714098264727068026730424033578827886916761701429264950573899186177


The goal of this notebook is to show that there are also short proofs (though not as short) for the primality of primes such as the 2 Mersenne numbers above.

## Traditional primality algorithm
The traditional way of checking that a number $n$ is prime is by making sure it cannot be evenly divided by smaller numbers. Actually, checking up to $\sqrt n$ is enough.

In [2]:
# Returns True if n is prime.
def is_prime(n):
    if n <= 2: return False
    if n == 2: return True
    if n % 2 == 0: return False
    i = 3
    while (i*i <= n):
        if n % i == 0:
            return False
        i += 2
    return True

print('List of prime numbers:\n')
for n in range(1, 1000):
    if is_prime(n):
        print(n, end=' ')
print('...')

List of prime numbers:

3 5 7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71 73 79 83 89 97 101 103 107 109 113 127 131 137 139 149 151 157 163 167 173 179 181 191 193 197 199 211 223 227 229 233 239 241 251 257 263 269 271 277 281 283 293 307 311 313 317 331 337 347 349 353 359 367 373 379 383 389 397 401 409 419 421 431 433 439 443 449 457 461 463 467 479 487 491 499 503 509 521 523 541 547 557 563 569 571 577 587 593 599 601 607 613 617 619 631 641 643 647 653 659 661 673 677 683 691 701 709 719 727 733 739 743 751 757 761 769 773 787 797 809 811 821 823 827 829 839 853 857 859 863 877 881 883 887 907 911 919 929 937 941 947 953 967 971 977 983 991 997 ...


This works well for relatively small numbers but doesn't scale up, say for 100-digit numbers. Note that the time complexity is $\Theta(\sqrt n)$, that is __exponential on the number of digits__ of $n$.

## Fermat's little theorem

__Fermat's little theorem__

If $p$ is prime and $0 < a < p$ then

$$a^{p-1} = 1 \mod p$$

__Proof__

For any $a$

$$(a + 1)^{p} = {p\choose 0}a^p + {p\choose 1}a^{p-1} + {p\choose 2}a^{p-2} + \cdots + {p\choose p}a^0$$

Since $p$ is prime, most of the coefficients are divisible by $p$. Indeed for $1 \leq q \leq p-1$, ${p\choose q} = {{p!} \over {q!(p-q)!}}$ has $p$ in the numerator, but not in the denominator. Therefore we have

$$(a + 1)^{p} = a^p + 1 \mod p$$

For $a = 1$, we get $$2^p = 2 \mod p$$
and using induction on $a$, we get more generally $$a^p = a \mod p$$

If $(a, p) = 1$, $a$ has an inverse (mod p), as can be seen using the generalized Euclid's algorithm, and therefore, for $0 < a < p$

$$a^{p-1} = 1 \mod p$$

Let's check the theorem for a few values:

In [3]:
# Computes a**n mod m.
def pow_mod(a, n, m):
    if a >= m:
        a %= m
    if n == 0:
        return 1
    result = pow_mod(a, n // 2, m) ** 2
    if n % 2 == 1: 
        result = a * result
    result = result % m
    return result

def fermat_test(n, a):
    return pow_mod(a, n-1, n) == 1

def check_fermat(a, n):
    assert(fermat_test(n, a))
    print(a, ' ** (', n-1, ' - 1) = 1 mod ', n, sep='')
    
check_fermat(2, 3)
check_fermat(2, 97)
check_fermat(23, 59)
check_fermat(2, mersenne_521)
check_fermat(2, mersenne_607)

2 ** (2 - 1) = 1 mod 3
2 ** (96 - 1) = 1 mod 97
23 ** (58 - 1) = 1 mod 59
2 ** (6864797660130609714981900799081393217269435300143305409394463459185543183397656052122559640661454554977296311391480858037121987999716643812574028291115057150 - 1) = 1 mod 6864797660130609714981900799081393217269435300143305409394463459185543183397656052122559640661454554977296311391480858037121987999716643812574028291115057151
2 ** (531137992816767098689588206552468627329593117727031923199444138200403559860852242739162502265229285668889329486246501015346579337652707239409519978766587351943831270835393219031728126 - 1) = 1 mod 531137992816767098689588206552468627329593117727031923199444138200403559860852242739162502265229285668889329486246501015346579337652707239409519978766587351943831270835393219031728127


On the other hand:

In [4]:
composite = mersenne_521 * mersenne_607
composite_power = pow_mod(2, composite-1, composite)
assert(composite_power != 1)
print('n =', composite)
print('\nis composite. Indeed: 2^(n-1) mod n =\n')
print(composite_power)

n = 3646154850295011369707131011438711095400799139943170490872585628683549034362552065955809589514611470241298944167703929337528884908857116141935206466329731087514964112054543019336536216107629523597606330154669196064144182472739556974502462402438903115845725630946428943768540714098264727068026730424033578827886916761701429264950573899186177

is composite. Indeed: 2^(n-1) mod n =

117617898447960240594738066243890558744589682390176529192112418734563680630878699069868895603571073655147079382052381245794746597893046791796022644053827181234957536466206089534971894653069934981964122654984031075848787023021459140241103234472410417849613699642368118324583536948112226181224721873646572082806254147448685639776817059429343


Note that Fermat's test is very fast even for very large numbers. For some composite numbers, like the one above, it gives a very fast proof of compositeness that doesn't require knowledge of the factors.

## Pseudoprimes

Fermat's little theorem is remarkable not only in itself, but also because its converse is almost true. But not quite as we are going to see.

__Poulet numbers__

Does $$2^{n-1} = 1 \mod n$$ imply that $n$ is prime?

Unfortunately, this is not true in general. Such numbers, composite but verifying the identity above, are called Poulet numbers.

In [5]:
def is_poulet(n):
    return fermat_test(n, 2) and not is_prime(n)

# Returns the list of pseudoprimes in an interval, as well
# as the number of primes in that interval.
def pseudoprimes(start, end, is_pseudoprime):
    num_primes = 0
    pseudoprimes = []
    for n in range(start, end + 1):
        if is_prime(n):
            num_primes += 1
        elif is_pseudoprime(n):
            pseudoprimes.append(n)
    return pseudoprimes, num_primes

def study_pseudoprimes(start, end, is_pseudoprime, name):
    pseudoprimes_list, num_primes = pseudoprimes(start, end, is_pseudoprime)
    print('Interval[', start, ',', end, ']')
    print('  ', num_primes, 'primes')
    print('  ',  len(pseudoprimes_list), name, ':', pseudoprimes_list)

print('Poulet numbers\n')            
study_pseudoprimes(2, 1000, is_poulet, 'poulet numbers')
print()
study_pseudoprimes(1000000, 1010000, is_poulet, 'poulet numbers')

Poulet numbers

Interval[ 2 , 1000 ]
   167 primes
   3 poulet numbers : [341, 561, 645]

Interval[ 1000000 , 1010000 ]
   753 primes
   1 poulet numbers : [1004653]


While it looks like Poulet numbers are rare and it is a good heuristic (and it seems to get better for larger numbers) it is not a __proof__ of primality.

__2-3 Pseudoprimes__

Our next attempt is to extend the test, and use 2 and 3 (and not just 2). 

Does $$2^{n-1} = 1 \mod n$$ and $$3^{n-1} = 1 \mod n$$ imply that $n$ is prime?

Things get better but, unfortunately, the answer is no again.

In [6]:
def is_2_3_pseudoprime(n):
    return fermat_test(n, 2) and fermat_test(n, 3) and not is_prime(n)

print('2-3 pseudoprimes\n')

study_pseudoprimes(2, 1000, is_2_3_pseudoprime, '2-3 pseudoprimes')
study_pseudoprimes(1000000, 1010000, is_2_3_pseudoprime, '2-3 pseudoprimes')
study_pseudoprimes(1000000, 1100000, is_2_3_pseudoprime, '2-3 pseudoprimes')

2-3 pseudoprimes

Interval[ 2 , 1000 ]
   167 primes
   0 2-3 pseudoprimes : []
Interval[ 1000000 , 1010000 ]
   753 primes
   0 2-3 pseudoprimes : []
Interval[ 1000000 , 1100000 ]
   7216 primes
   4 2-3 pseudoprimes : [1024651, 1033669, 1050985, 1082809]


We could try to refine this approach, as it seems to produce better and better heuristics. But, again, it doesn't seem that we will get a criteria of primality.

## A criteria of primality

__Theorem__

$n$ is prime if and only if, there is $g$ such that, modulo $n$,

\begin{align*}
g^1 \neq 1, g^2 \neq 1, g^3 \neq 1, \ldots, g^{n-2} \neq 1 \\
g^{n-1} = 1
\end{align*}

Such a $g$ is called a __witness__ of primality of $n$.

__Proof__

We will prove this at the end of the notebook, but let's verify here the theorem on some examples.

__Examples__

In [7]:
def get_power_sequence(n, g):
    sequence = []
    term = g
    for i in range(1, n):
        sequence.append(term)
        term *= g
        term = term % n
    return sequence
    
def print_power_sequence(n, g):
    print(g, ' ** i (mod ', n, ') : ', end='', sep='')
    sequence = get_power_sequence(n, g)
    for term in sequence:
        print(term, end=' ')
    print('...')

def get_primality_witnesses(n):
    witnesses = []
    for a in range(1, n):
        sequence = get_power_sequence(n, a)
        has_witness = True
        for i in range(1, n):
            if not i in sequence:
                has_witness = False
                break
        if has_witness:
            witnesses.append(a)
    return witnesses

def print_primality_witnesses(n):
    print('Witnesses for', n, ':', get_primality_witnesses(n))

__11 is prime__

In [8]:
n = 11
for a in range(1, n):
    print_power_sequence(n, a)

print()
print_primality_witnesses(n)

1 ** i (mod 11) : 1 1 1 1 1 1 1 1 1 1 ...
2 ** i (mod 11) : 2 4 8 5 10 9 7 3 6 1 ...
3 ** i (mod 11) : 3 9 5 4 1 3 9 5 4 1 ...
4 ** i (mod 11) : 4 5 9 3 1 4 5 9 3 1 ...
5 ** i (mod 11) : 5 3 4 9 1 5 3 4 9 1 ...
6 ** i (mod 11) : 6 3 7 9 10 5 8 4 2 1 ...
7 ** i (mod 11) : 7 5 2 3 10 4 6 9 8 1 ...
8 ** i (mod 11) : 8 9 6 4 10 3 2 5 7 1 ...
9 ** i (mod 11) : 9 4 3 5 1 9 4 3 5 1 ...
10 ** i (mod 11) : 10 1 10 1 10 1 10 1 10 1 ...

Witnesses for 11 : [2, 6, 7, 8]


__15 is not prime__

In [9]:
n = 15
for a in range(1, n):
    print_power_sequence(n, a)

print()
print_primality_witnesses(n)

1 ** i (mod 15) : 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...
2 ** i (mod 15) : 2 4 8 1 2 4 8 1 2 4 8 1 2 4 ...
3 ** i (mod 15) : 3 9 12 6 3 9 12 6 3 9 12 6 3 9 ...
4 ** i (mod 15) : 4 1 4 1 4 1 4 1 4 1 4 1 4 1 ...
5 ** i (mod 15) : 5 10 5 10 5 10 5 10 5 10 5 10 5 10 ...
6 ** i (mod 15) : 6 6 6 6 6 6 6 6 6 6 6 6 6 6 ...
7 ** i (mod 15) : 7 4 13 1 7 4 13 1 7 4 13 1 7 4 ...
8 ** i (mod 15) : 8 4 2 1 8 4 2 1 8 4 2 1 8 4 ...
9 ** i (mod 15) : 9 6 9 6 9 6 9 6 9 6 9 6 9 6 ...
10 ** i (mod 15) : 10 10 10 10 10 10 10 10 10 10 10 10 10 10 ...
11 ** i (mod 15) : 11 1 11 1 11 1 11 1 11 1 11 1 11 1 ...
12 ** i (mod 15) : 12 9 3 6 12 9 3 6 12 9 3 6 12 9 ...
13 ** i (mod 15) : 13 4 7 1 13 4 7 1 13 4 7 1 13 4 ...
14 ** i (mod 15) : 14 1 14 1 14 1 14 1 14 1 14 1 14 1 ...

Witnesses for 15 : []


__337 is prime__

In [10]:
print_primality_witnesses(337)

Witnesses for 337 : [10, 15, 19, 20, 22, 23, 29, 31, 33, 34, 44, 45, 46, 51, 53, 60, 61, 67, 68, 70, 71, 73, 80, 83, 87, 89, 90, 93, 99, 101, 106, 109, 114, 116, 118, 120, 124, 130, 132, 134, 139, 143, 151, 152, 154, 160, 161, 166, 171, 176, 177, 183, 185, 186, 194, 198, 203, 205, 207, 213, 217, 219, 221, 223, 228, 231, 236, 238, 244, 247, 248, 250, 254, 257, 264, 266, 267, 269, 270, 276, 277, 284, 286, 291, 292, 293, 303, 304, 306, 308, 314, 315, 317, 318, 322, 327]


__341 is not prime__

Remember that 341 is a Poulet number and therefore a pseudoprime. It is therefore comforting that it does not have any witness.

In [11]:
print_primality_witnesses(341)

Witnesses for 341 : []


## Lucas Primality Test

This criteria of primality, while interesting, doesn't help in our goal of proving that PRIMES is in NP. Indeed, to apply the criteria we would need to perform computations on all the numbers up to the number we want to prove prime. But there are exponentially many of them.

Lucas Primality Test is a primality criteria that shows that in fact, we will only need to test the values of $g^k$ for a few $k$'s, and more precisely for a polynomial number of them.

__Theorem__ $n$ is prime if, and only if, there is $g$ such that:
1. $g^{n-1} = 1 \mod n$
2. for every prime factor $q$ of $n-1$, $g^{(n-1)/q} \neq 1 \mod n$

__Proof__

Note that this theorem is a version of the characterization of primes, with a weaker condition in 2. 

We assume that all computations are done mod $n$.

We need to prove that if $g^{n-1} = 1$ and $g^i = 1$, for some $0< i < n-1$, then there is a prime factor $q$ of $n-1$ such that $g^{(n-1)/q} = 1$.

Since $g^{n-1} = 1$, the sequence $g^k$ has period $r$, either $n-1$, or a proper factor of $n-1$. Because, $g^i = 1$ for some $0<i<n-1$, the period needs to be a proper factor $r$ of $n-1$.

Therefore, $g^{r} = 1$ for $r = {{n-1}\over{mq}}$, with $m >= 1$ and $q$ prime. 

Since $g^{r} = 1$, then $g^{rm} = 1$ therefore $g^{(n-1)/q} = 1$.


## Pratt's Theorem

__Theorem__ PRIMES is in NP

__Proof__ 

__Certificate__
To produce a certificate of primality for $n$, write down a factorization of $n - 1$ into prime numbers, as well as a witness of primality for $n$. Recursively add, to the certificate, certificates of primality for the prime numbers in the factorization. Stop the recursion at the numbers 2 and 3, which are obviously prime.

__Length__
Consider the certificate tree with $n$ as the root, the children of a node $m$ being the prime factors of $m-1$, and having as leaves 2's and 3's. Note that each non leaf node has > 1 children. This implies that the tree, which has $O(\log n)$ leaves, has $O(\log n)$ nodes. Each node stands for a number $\leq n$ and therefore can be represented with $O(\log n)$ bits. In total we only need $O(\log^2 n)$ bits to represent this tree. For the $O(\log n)$ witnesses, one per node, $O(\log^2 n)$ bits are again sufficient. Therefore the certificate's length is polynomial in $\log n$.

__Running Time__
For each node, we need to check the factorization of a $O(\log n)$-bit number. This can be done in $O(\log^2 n)$ time with the standard multiplication algorithm. Since there are $O(\log n)$ nodes, we can check all the factorizations in $O(\log^3 n)$ time. As for witnesses, we need to check $O(\log n)$ of them, and each check requires a modular exponentiation which can be done $O(\log^3 n)$ time. Therefore the total time for verifying witnesses is $O(\log^4 n)$. In conclusion, the certificate can be verified in polynomial time.

## Primality Verifier

We present now a function that allows to verify primality certificates.

In [12]:
# Verifies that (factors, witnesses) is a valid primality certificate for n.
# The indentation helps readability, showing the recursion level in the output.
# The verified_set is the set of primes that have been already verified, so we 
# don't verify them again.
def verify_primality_certificate(n, factors, witnesses, indentation='', verified_set=[]):
    if n == 2 or n == 3:
        print(indentation, n, "is a known prime")
        return True
    if n in verified_set:
        print(indentation, n, "already verified")
        return True
    print(indentation, "verifying", n, "...")
    factors_for_n = factors[n]
    # Verify product.
    product = 1
    for f in factors_for_n:
        product *= f
    if product != n - 1:
        print(indentation, product, '!=', n - 1)
        return False
    print(indentation, "product checked for", n, '- 1')
    # Verify criteria.
    witness = witnesses[n]
    if pow_mod(witness, n-1, n) != 1:
        return False
    for f in factors_for_n:
        if pow_mod(witness, (n-1)//f, n) == 1:
            print('Oops', witness, n, (n-1)//f, n)
            return False
    print(indentation, "witness checked for", n)
    # Recurse.
    print(indentation, 'recursing on factors', factors_for_n, 'of', n, '- 1')
    for f in factors_for_n:
        if not verify_primality_certificate(f, factors, witnesses, indentation + '  ', verified_set):
            return False
    print(indentation, '==>', n, "is prime")
    verified_set.append(n)
    return True

__Verifying 31__

In [13]:
n = 31

# (factors, witnesses) is the certificate of primality for 31.
factors = {n:[2,3,5], 3:[2], 5:[2,2]}
witnesses = {n:3, 3:2, 5:2}

verify_primality_certificate(n, factors, witnesses, '', [])

 verifying 31 ...
 product checked for 31 - 1
 witness checked for 31
 recursing on factors [2, 3, 5] of 31 - 1
   2 is a known prime
   3 is a known prime
   verifying 5 ...
   product checked for 5 - 1
   witness checked for 5
   recursing on factors [2, 2] of 5 - 1
     2 is a known prime
     2 is a known prime
   ==> 5 is prime
 ==> 31 is prime


True

__Verifying $2^{89} - 1$__

In [14]:
mersenne_89 = 2**89 - 1

# (factors, witnesses) is the certificate of primality for mersenne_89.

# Maps n to the factors of n-1.
# The n's are the primes needed in the verification.
factors = {mersenne_89:[2,3,5,17,23,89,353,397,683,2113,2931542417],
           3: [2],
           5: [2,2],
           7: [2,3],
           11: [2,5],
           17: [2,2,2,2],
           23: [2,11],
           29: [2,2,7],
           31: [2,3,5],
           89: [2,2,2,11],
           239: [2,7,17],
           353: [2,2,2,2,2,11],
           397: [2,2,3,3,11],
           683: [2,11,31],
           1451: [2,5,5,29],
           1913: [2,2,2,239],
           2113: [2,2,2,2,2,2,3,11],
           8707: [2,3,1451],
           2931542417: [2,2,2,2,11,1913,8707]}

# Maps n to its witness for Fermat's test.
# The n's are the primes needed in the verification.
witnesses = {mersenne_89:3, 
             3: 2, 
             5: 2, 
             7: 3,
             11: 2, 
             17: 3, 
             23: 5, 
             29: 2,
             31: 3, 
             89: 3, 
             239: 7,
             353: 3, 
             397: 5, 
             683: 5,
             1451: 2,
             1913: 3,
             2113: 5, 
             8707: 5,
             2931542417: 3}

verify_primality_certificate(mersenne_89, factors, witnesses, '', [])

 verifying 618970019642690137449562111 ...
 product checked for 618970019642690137449562111 - 1
 witness checked for 618970019642690137449562111
 recursing on factors [2, 3, 5, 17, 23, 89, 353, 397, 683, 2113, 2931542417] of 618970019642690137449562111 - 1
   2 is a known prime
   3 is a known prime
   verifying 5 ...
   product checked for 5 - 1
   witness checked for 5
   recursing on factors [2, 2] of 5 - 1
     2 is a known prime
     2 is a known prime
   ==> 5 is prime
   verifying 17 ...
   product checked for 17 - 1
   witness checked for 17
   recursing on factors [2, 2, 2, 2] of 17 - 1
     2 is a known prime
     2 is a known prime
     2 is a known prime
     2 is a known prime
   ==> 17 is prime
   verifying 23 ...
   product checked for 23 - 1
   witness checked for 23
   recursing on factors [2, 11] of 23 - 1
     2 is a known prime
     verifying 11 ...
     product checked for 11 - 1
     witness checked for 11
     recursing on factors [2, 5] of 11 - 1
       2 is a k

True

## Proof of the criteria of primality

__Note:__ this section assumes some knowledge of Group Theory and Number Theory.

### Basic Notions

__Euler's totient function__

$\phi(n)$ denotes the number of elements between 1 and $n$, that are relatively prime to $n$.

__Group $Z^*_n$__

$Z^*_n$ is the multiplicative group of integers modulo n. It has $\phi(n)$ elements.

__Lagrange's Theorem__

If a group is finite, the order (that is the number of elements) of any of its subgroups divides the order of the group.

__Euler's theorem__

If $(a, n) = 1$, then $a^{\phi(n)} = 1$. This can be easily derived from Lagrange's theorem and can be seen as a generalization of Fermat's little theorem.

__Order of an element__

The order of an element $g$ of a group is, by definition, the order of the subgroup generated by $g$, that is the smallest $k \geq 1$ such that $g^k = 1$.

In [15]:
def gcd(a, b):
    if b == 0:
        return a
    return gcd(b, a % b)

def phi(n):
    result = 0
    for a in range(1, n+1):
        if gcd(a, n) == 1:
            result += 1
    return result

def order(g, n):
    assert(gcd(g, n) == 1)
    pow = 1
    g_pow = g
    while g_pow != 1:
        g_pow = (g_pow * g) % n 
        pow += 1
    return pow

Let's check Euler's theorem on some value

In [16]:
def check_euler(n):
    for a in range(n):
        if gcd(a, n) == 1:
            assert(pow_mod(a, phi(n), n) == 1)

for n in range(2, 101):
    check_euler(n)
print('Euler\'s theorem checked between 2 and 100')
    
check_euler(341)
print('Euler\'s theorem checked on 341')

Euler's theorem checked between 2 and 100
Euler's theorem checked on 341


### Supporting Theorems

__Theorem - Sum of Totients__

For any $n \geq 1$, $$\sum_{d|n}{\phi(d)} = n$$

Let's try this on $n = 12$.

In [17]:
n = 13
for a in range(1, n):
    if (n-1) % a == 0:
        print('  phi(', a, ') = ', phi(a), sep='')

  phi(1) = 1
  phi(2) = 1
  phi(3) = 2
  phi(4) = 2
  phi(6) = 2
  phi(12) = 4


So indeed $\phi(1) + \phi(2) + \phi(3) + \phi(4) + \phi(6) + \phi(12) = 12$. Let's prove this in general.

__Proof__

Consider the sequence ${{1}\over{n}}, {{2}\over{n}}, {{3}\over{n}}, \ldots, {{n-1}\over{n}}, {{n}\over{n}}$ and simplify the fractions. Associate each $1 \leq i \leq n$ to the $d$ such that ${i\over n} = {k \over d}$ with $(k, d) = 1$.

__Theorem - Roots of a polynomial__

For $p$ prime, polynomials of degree $d$ have at most $d$ distinct solutions in $Z^*_p$.

__Proof__

TODO

### Proof of the criteria

All the computations will be done mod $n$.

If $n$ is composite, there are 2 cases:
1. $(g, n) \neq 1$: then $g^k \neq 1$, for any $k$. In particular $g^{n-1} \neq 1$ and therefore $g$ is not a witness.
2. $(g, n) = 1$: then $g^{\phi(n)} = 1$. Since $\phi(n) < n - 1$, $g$ is not a witness.

Let's assume that $n$ is prime, and let's call it $p$ from now on. 

We will show a stronger version of the criteria: the number of elements of order $d$ of $Z^*_p$, where $d$ divides $p-1$, is exactly $\phi(d)$. This implies that there are $\phi(p-1) > 0$ elements of order $p-1$, and therefore that there is at least 1 witness.

Let's check this is true on an example.

In [18]:
p = 13
print('modulo', p)
for g in range(1, p):
    print('  order(', g, ') = ', order(g, p), sep='')

print('\ntotients:')
for a in range(1, p):
    if (p-1) % a == 0:
        print('  phi(', a, ') = ', phi(a), sep='')

modulo 13
  order(1) = 1
  order(2) = 12
  order(3) = 3
  order(4) = 6
  order(5) = 4
  order(6) = 12
  order(7) = 12
  order(8) = 4
  order(9) = 3
  order(10) = 6
  order(11) = 12
  order(12) = 2

totients:
  phi(1) = 1
  phi(2) = 1
  phi(3) = 2
  phi(4) = 2
  phi(6) = 2
  phi(12) = 4


Note that the assertion above is true for $p = 13$. For example, there are 4 elements of order 12 (2, 6, 7, 11) and indeed $\phi(12) = 4$.

Continuing the proof, note that any element $g$ generates a subgroup of $Z^*_p$ and therefore, using Lagrange's theorem, the order of any $g$ divides $p-1$.

Let $d$ be a divisor of $p-1$. According to the second supporting theorem, there are at most $d$ elements such as $g^d = 1$, that is there are at most $d$ elements of order $d$. Let $g$ be such an element, if it exists. Then $g^1, g^2, \ldots, g^{d}$ are different and they are all the solutions of $g^d = 1$. But only $\phi(d)$ of them have order $d$, since if $k$ is such that $(k, p-1)$

We have proved that:
* if $d$ does not divide $p-1$, $d$ cannot be the order of any $g$.
* If $d$ divides $p-1$, then there are at most $\phi(d)$ elements of order $d$.

Because of the first supporting theorem, this means that there are exactly  $\phi(d)$ elements of order $d$. 

QED